In [ ]:
import pandas as pd
import numpy as np
import csv
import json
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_name = '/content/drive/My Drive/Product_Classification/Data/data_output.csv'
raw_data = pd.read_csv(filename)
raw_data.drop(['Unnamed: 0'], axis = 1, inplace = True)
print(raw_data.shape)
print(raw_data.columns)

(14612, 16)
Index(['ProductId', 'ProductName', 'Branch', 'level_1', 'level_2', 'level_3',
       'Image', 'ProductUrl', 'PricePerItem', 'PriceMode', 'HasMultiBuyDeal',
       'MultiBuyDeal', 'MultiBuyBasePrice', 'MultiBuyPrice',
       'MultiBuyQuantity', 'PromoBadgeImageLabel'],
      dtype='object')


In [ ]:
raw_data.head(5)

,ProductId,ProductName,Branch,level_1,level_2,level_3,Image,ProductUrl,PricePerItem,PriceMode,HasMultiBuyDeal,MultiBuyDeal,MultiBuyBasePrice,MultiBuyPrice,MultiBuyQuantity,PromoBadgeImageLabel
0,5000961-EA-000NW,Johnson's Baby Lotion,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care,https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5000961_ea_000...,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
1,5000962-EA-000NW,Johnson's Baby Oil,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Bathing & Skin Care,https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5000962_ea_000...,4.89,ea,False,NaN,4.89,NaN,NaN,NaN
2,5000963-EA-000NW,SMA Balanced Nutrition From Birth Infant Formula,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Baby Formula & Toddler Food,https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5000963_ea_000...,17.99,ea,False,NaN,17.99,17.99,1.0,Saver
3,5000965-EA-000NW,Johnson's Baby Powder,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Nappies & Changing,https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5000965_ea_000...,2.99,ea,False,NaN,2.99,2.99,1.0,Saver
4,5000966-EA-000NW,Johnson's Baby Powder,New World Mt Roskill,"Baby, Toddler & Kids",Baby Care,Nappies & Changing,https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5000966_ea_000...,4.89,ea,False,NaN,4.89,NaN,NaN,NaN


In [ ]:
raw_data['PromoBadgeImageLabel'] = raw_data['PromoBadgeImageLabel'].astype('str') 

In [ ]:
raw_data['leibie'] = raw_data.apply(lambda row: 'regular' if row['PromoBadgeImageLabel'] == 'nan' else 'special', axis=1)

In [ ]:
print(raw_data.columns)

Index(['ProductId', 'ProductName', 'Branch', 'level_1', 'level_2', 'level_3',
       'Image', 'ProductUrl', 'PricePerItem', 'PriceMode', 'HasMultiBuyDeal',
       'MultiBuyDeal', 'MultiBuyBasePrice', 'MultiBuyPrice',
       'MultiBuyQuantity', 'PromoBadgeImageLabel', 'leibei', 'leibie'],
      dtype='object')


In [ ]:
new_data = raw_data[['ProductName', 'PricePerItem', 'Branch', 'leibie', 'level_1', 'Image', 'ProductUrl']]
new_data.rename(columns={'ProductName': 'name', 'PricePerItem': 'price', 'Branch': 'fdName', 'level_1': 'cat', 'Image': 'images', 'ProductUrl':'info'}, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
new_data['url'] = new_data['info']
new_data.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,name,price,fdName,leibie,cat,images,info,url
0,Johnson's Baby Lotion,4.89,New World Mt Roskill,regular,"Baby, Toddler & Kids",https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5000961_ea_000...,www.ishopnewworld.co.nz/product/5000961_ea_000...
1,Johnson's Baby Oil,4.89,New World Mt Roskill,regular,"Baby, Toddler & Kids",https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5000962_ea_000...,www.ishopnewworld.co.nz/product/5000962_ea_000...
2,SMA Balanced Nutrition From Birth Infant Formula,17.99,New World Mt Roskill,special,"Baby, Toddler & Kids",https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5000963_ea_000...,www.ishopnewworld.co.nz/product/5000963_ea_000...
3,Johnson's Baby Powder,2.99,New World Mt Roskill,special,"Baby, Toddler & Kids",https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5000965_ea_000...,www.ishopnewworld.co.nz/product/5000965_ea_000...
4,Johnson's Baby Powder,4.89,New World Mt Roskill,regular,"Baby, Toddler & Kids",https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5000966_ea_000...,www.ishopnewworld.co.nz/product/5000966_ea_000...


In [ ]:
new_data.to_csv('data_output_follow_countdown.csv')

# Data recrawling

In [ ]:
## Find duplicate products in each store

# 1. Delete exactly the same data (same with each variable)
raw_data.drop_duplicates(subset = None, keep = 'first', inplace = True)
print(raw_data.shape)

NameError: ignored

In [ ]:
# 2. Delete 'Featured' 
# Since all 'Featured' products are classified in other categories
raw_data = raw_data[raw_data['level_1'] != 'Featured']
print(raw_data.shape)

(14488, 16)


In [ ]:
# 3. It is found that one product is categorized in different categories
raw_data['ProductId'].value_counts()

5223555-EA-000NW    2
5018420-EA-000NW    2
5086046-EA-000NW    2
5010428-EA-000NW    2
5203953-EA-000NW    2
                   ..
5002240-EA-000NW    1
5086576-EA-000NW    1
5241873-EA-000NW    1
5250520-EA-000NW    1
5083496-EA-000NW    1
Name: ProductId, Length: 13136, dtype: int64

In [ ]:
raw_data[raw_data['ProductId'] == '5200248-EA-000NW']

,ProductId,ProductName,Branch,level_1,level_2,level_3,Image,ProductUrl,PricePerItem,PriceMode,HasMultiBuyDeal,MultiBuyDeal,MultiBuyBasePrice,MultiBuyPrice,MultiBuyQuantity,PromoBadgeImageLabel
2623,5200248-EA-000NW,Nuttelex Coconut Oil Spread,New World Mt Roskill,"Chilled, Frozen & Desserts",Dairy & Eggs,Butter & Spreads,https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5200248_ea_000...,2.99,ea,True,Buy 2 for 5.00.,2.99,5.0,2.0,Multibuy Super Saver
5478,5200248-EA-000NW,Nuttelex Coconut Oil Spread,New World Mt Roskill,Fresh Foods & Bakery,Dairy & Eggs,Butter & Spreads,https://a.fsimg.co.nz/product/retail/fan/image...,www.ishopnewworld.co.nz/product/5200248_ea_000...,2.99,ea,True,Buy 2 for 5.00.,2.99,5.0,2.0,Multibuy Super Saver


In [ ]:
# Delete duplicate product
duplicate_products = raw_data[raw_data.duplicated(subset = ['ProductId'], keep = 'first')]

# Convert dataframe to csv
duplicate_products.to_csv('duplicated_data.csv')

# Convert csv to json
with open('duplicated_data.csv') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

with open('duplicated_data.json', 'w') as f:
    json.dump(rows, f)

# Category renewal

In [ ]:
## Update category for duplicated data